# Day 11: Seating System
---
Soalnya capek karena di bagian kedua harus refaktor kembali fungsi yang dipakai di bagian sebelumnya.

In [1]:
inputs = None
with open("input.txt") as file:
    inputs = file.read().strip()

inputs[:30]

'LLL.LLL.LLLLL.LLLLLLLLLLLLLLLL'

---
## Classes

Di sini aku buat 2 *class*, yang pertama *class* ```Grid``` untuk membungkus struktur data *grid* atau *array* 2 dimensi, dan kedua baru *class* untuk menjalankan siklus perubahan kondisinya.

*Class* ```Grid``` nantinya dapat digunakan lagi untuk soal yang menggunakan struktur data 2 dimensi dengan cara diekspor ke file ```.py```

In [2]:
class Grid:
    directions = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))

    def __init__(self, grid = ((),)):
        self.grid = tuple(tuple(row) for row in grid)
        self.max_row = len(grid)
        self.max_col = len(grid[0])

    def __get(self, row, col):
        if 0 <= row < self.max_row and 0 <= col < self.max_col:
            return self.grid[row][col]
        return None

    def to_list(self):
        return list(list(row) for row in self.grid)

    def get_adjacents_cells(self, row, col):
        return self.get_all_directions_cells(row, col)
    
    def get_all_directions_cells(self, row, col, lookup_func=lambda x: True):
        result = []
        for dy, dx in self.directions:
            y = row
            x = col
            while True:
                y += dy
                x += dx
                subresult = self.__get(y, x)
                if subresult == None or lookup_func(subresult):
                    result.append(subresult)
                    break
        return result

    def enumerate(self):
        for row in range(self.max_row):
            for col in range(self.max_col):
                yield row, col, self.grid[row][col]

    def __str__(self):
        return "\n".join("".join(*zip(*row)) for row in self.grid)

    def __repr__(self):
        return str(self)

    def __eq__(self, other):
        return (
            self.__class__ == other.__class__ and
            self.grid == other.grid
        )


In [3]:
class SeatCycle:
    first_rule = (lambda x: x in "L#.", 4)
    second_rule = (lambda x: x in "L#", 5)
    def __init__(self, inputs):
        self.initial_state = Grid([[ch for ch in row] for row in inputs.strip().split("\n")])

    def __next_cycle(self, previous=None, lookup_func=lambda x: True, tolerance=4):
        previous = self.initial_state if previous == None else previous
        result = previous.to_list()
        for row, col, value in previous.enumerate():
            if value == "L":
                if previous.get_all_directions_cells(row, col, lookup_func).count("#") == 0:
                    result[row][col] = "#"
            elif value == "#":
                if previous.get_all_directions_cells(row, col, lookup_func).count("#") >= tolerance:
                    result[row][col] = "L"
        return Grid(result)
    
    def get_n_cycle_state(self, n, rule=first_rule):
        result = self.initial_state
        lookup_func, tolerance = rule
        for i in range(n):
            result = self.__next_cycle(result, lookup_func, tolerance)
        return result
    
    def get_stable_state(self, rule=first_rule):
        prev_state = self.initial_state
        lookup_func, tolerance = rule
        while True:
            next_state = self.__next_cycle(prev_state, lookup_func, tolerance)
            if (next_state == prev_state):
                break
            prev_state = next_state
        return prev_state

---
## Unit Testing

Karena contoh soalnya panjang banget, unit testingnya juga jadi panjang.

In [4]:
test_sc = SeatCycle(
"""
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""
)

In [5]:
# After one round of these rules, every seat in the example layout becomes occupied:

test_sc.get_n_cycle_state(1)

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##

In [6]:
# After a second round, the seats with four or more occupied adjacent seats become empty again:

test_sc.get_n_cycle_state(2)

#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##

In [7]:
# At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state!
# Once people stop moving around, you count 37 occupied seats.

str(test_sc.get_stable_state()).count("#") == 37

True

In [8]:
# For example, the empty seat below would see eight occupied seats:

test_sc2 = SeatCycle(
"""
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
"""
)

str(test_sc2.initial_state.get_all_directions_cells(4, 3, lambda x: x in "L#")).count("#") == 8

True

In [9]:
# The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

test_sc3 = SeatCycle(
"""
.............
.L.L.#.#.#.#.
.............
"""
)

sight = test_sc3.initial_state.get_all_directions_cells(1, 1, lambda x: x in "L#")
str(sight).count("L") == 1 and str(sight).count("#") == 0

True

In [10]:
# The empty seat below would see no occupied seats:

test_sc4 = SeatCycle(
"""
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
"""
)

str(test_sc4.initial_state.get_all_directions_cells(3, 3, lambda x: x in "L#")).count("#") == 0

True

In [11]:
# Again, at this point, people stop shifting around and the seating area reaches equilibrium.
# Once this occurs, you count 26 occupied seats.

test_sc5 = SeatCycle(
"""
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""
)

str(test_sc5.get_stable_state(rule=SeatCycle.second_rule)).count("#") == 26

True

---
## Part 1
Bagian pertama cukup gunakan ```while``` untuk mencari *state* yang sama setelah ```next_cycle``` dijalankan.

In [12]:
sc = SeatCycle(inputs)

%time str(sc.get_stable_state()).count("#")

Wall time: 4.82 s


2265

---
## Part 2
Bagian kedua hanya merubah kriterianya dengan mengganti ```lookup_func``` dan ```tolerance```. Aku buat 2 *class variable* untuk menampung aturan yang aku masukkan ke dalam fungsinya (*default* menggunakan aturan pertama ```SeatCycle.first_rule```).

In [13]:
%time str(sc.get_stable_state(rule=SeatCycle.second_rule)).count("#")

Wall time: 6.25 s


2045